In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key

# os.environ['HOPSWORKS_API_KEY'] = 'your_api_key'

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1048724
2024-09-23 21:35:33,649 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "nigeria", "city": "abuja", "street": "us-embassy", "aqicn_url": "https://api.waqi.info/feed/@13449", "latitude": 9.6, "longitude": 8.0}'

In [4]:
air_quality_fg = fs.get_feature_group(
    name = "air_quality",
    version = 1,
)

weather_fg = fs.get_feature_group(
    name = "weather",
    version = 1,
)

In [5]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df['pm25'] = aq_today_df['pm25'].astype(float)

In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float64       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 180.0+ bytes


In [7]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

hourly_df

Coordinates 9.5°N 8.0°E
Elevation 737.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
date,,,,
2024-09-23 00:00:00,21.450001,0.0,5.860375,169.380402
2024-09-23 01:00:00,21.250000,2.0,6.480000,180.000000
2024-09-23 02:00:00,21.049999,2.0,6.924738,188.972549
2024-09-23 03:00:00,20.850000,2.0,6.569383,189.462250
2024-09-23 04:00:00,20.549999,0.1,4.802999,167.005386
...,...,...,...,...
2024-10-02 19:00:00,21.900000,0.2,4.735060,261.253906
2024-10-02 20:00:00,21.450001,0.2,5.692099,235.304779
2024-10-02 21:00:00,21.049999,0.2,7.704336,217.405426


In [8]:
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-09-23,26.799999,0.0,8.913181,223.363464,abuja
1,2024-09-24,28.299999,0.0,8.427383,109.983192,abuja
2,2024-09-25,24.400000,0.3,8.473393,167.735245,abuja
3,2024-09-26,28.900000,0.0,8.825508,168.231735,abuja
4,2024-09-27,27.450001,0.0,2.305125,308.659821,abuja
5,2024-09-28,26.750000,0.1,5.116561,140.710617,abuja
6,2024-09-29,27.350000,0.1,2.968636,165.963730,abuja
7,2024-09-30,27.500000,0.0,6.214563,169.992081,abuja
8,2024-10-01,27.900000,0.1,6.489992,183.179764,abuja
9,2024-10-02,29.650000,0.0,5.154416,167.905258,abuja


In [9]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


In [10]:
aq_today_df

,pm25,country,city,street,date,url
0,57.0,nigeria,abuja,us-embassy,2024-09-23,https://api.waqi.info/feed/@13449


In [11]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float64       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 180.0+ bytes


In [12]:
air_quality_fg.insert(aq_today_df)

2024-09-23 21:35:39,115 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1048724/fs/1040451/fg/1204590


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1048724/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x12aff5e7bf0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 621569
         }
       },
       "result": {
         "observed_value": 57.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-09-23T08:35:39.000115Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsu

In [13]:
weather_fg.insert(daily_df)

2024-09-23 21:35:49,926 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1048724/fs/1040451/fg/1206605


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1048724/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x12aff4e9b80>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 622594
         }
       },
       "result": {
         "observed_value": 2.3051247596740723,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-09-23T08:35:49.000926Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column